# 18.S190/6.S090 Problem Set 1

Due Friday 2/14 at **11:59pm**; 20% penalty if it is turned in within 24 hours, and after that late psets will not be accepted.   Submit in PDF format: a decent-quality scan/image of any handwritten solutions (e.g. get a scanner app on your phone or use a tablet), combined with a PDF printout of your Jupyter notebook showing your code and (clearly labeled) results.

**TO GENERATE A PDF OF YOUR JUPYTER NOTEBOOK:** In the Jupyter client (e.g. the [JupyterLab Desktop](https://github.com/jupyterlab/jupyterlab-desktop) app), in the *File* pull-down menu, select *Save and Export Notebook As*, and then select the *HTML* format (not PDF, which may require special software).  Then open the downloaded HTML file with your favorite browser, and use the browser's *Print* function to generate the PDF file.

## Problem 1 (5+5+6+4 points)

The [course notebook on finite differences](https://github.com/mitmath/numerical_hub/blob/fbcbf6adef724392624921c5a7cf8a9d53330347/notes/finite-differences.ipynb)
includes, without derivation, a mysterious four-line Julia function
called `stencil` that can compute finite-difference rules for
an arbitrary number of points.  The `stencil` function is reproduced below, in both Julia and Python.

In particular, if you want to compute
the $m$-th derivative of a smooth (analytic) scalar function $f(x)$
at $x_{0}$, it returns the weights $w_{k}$ of an $n$-point ($n>m$)
finite-difference rule from evaluating $f$ at points $x_{k}$ for
$k=1\ldots n$:
$$
f^{(m)}(x_{0})\approx\sum_{k=1}^{n}w_{k}f(x_{k})
$$
by solving the system of equations $Aw=e_{m+1}$, where $e_{j}\in\mathbb{R}^{n}$
is the Cartesian unit vector in the $j$-th direction and $A$ is
an $n\times n$ matrix with entries $A_{ij}=\frac{(x_{j}-x_{0})^{i-1}}{(i-1)!}$.

Here, you will analyze and derive this technique.

1. Let $x_{0}=0$. According to the notes, you can then compute $f^{(m)}(y)\approx\frac{1}{h^{m}}\sum_{k=1}^{n}w_{k}f(y+hx_{k})$
for an arbitrary point $y$ and an arbitrary step-size scaling factor
$h$ (which can be made smaller and smaller to reduce truncation errors; i.e. $h=\delta x$).
Derive this formula from the $f^{(m)}(x_{0})\approx \cdots$ formula above (via the chain rule and a change of variables).

2. Now evaluate it for $x_{0}=0$ (`0//1` in Julia for exact rational results)
and $x=[0,1,2,3]$ with $m=1$, i.e. using $n=4$ equally spaced points
$\ge x_{0}$ (a *higher-order* "forward-difference" formula). Use the resulting weights, in the formula scaled by
$h$ as above, to approximate the derivative $f'(1)$ for $f(x)=\sin(x)$,
and plot the relative error (compared to the exact derivative) as
a function of $h$ on a log–log scale, similar to the course notebook.
What power law in $h$ does the truncation error (approximately) seem
to follow? That is, what is the “order of accuracy”?

3. Derive the stencil equation $Aw=e_{m+1}$ above: write out the first
$n$ terms of the Taylor series (up to the $f^{(n-1)}$ derivative) for $f(x_{0}+\delta x)$, and try
to find a linear combination of this series evaluated at $\delta x=x_{k}-x_{0}$
for $k=1\ldots n$ in such a way that you obtain $f^{(m)}(x_{0})$.

4. Explain the output of `stencil` for $x =[-1,+1]$, $x_{0}=0$ (or `0//1` in Julia for exact rational results), and $m=0$?

In [ ]:
# in Julia:
function stencil(x::AbstractVector{<:Real}, x₀::Real, m::Integer)
    ℓ = 0:length(x)-1
    m in ℓ || throw(ArgumentError("invalid derivative order m"))
    A = @. (x' - x₀)^ℓ / factorial(ℓ)
    return A \ (ℓ .== m) # vector of weights w
end

In [ ]:
# in Python:
import numpy as np
from scipy.special import factorial

def stencil(x, x0, m):
    ℓ = np.arange(len(x))
    if m not in ℓ: raise ValueError("invalid derivative order m")
    ℓ_col = ℓ[:,np.newaxis]
    A = (np.asarray(x) - x0)**ℓ_col / factorial(ℓ_col)
    return np.linalg.solve(A, ℓ == m)

## Problem 2 (4+4+4+8 points)

Write a function `myexp(x)` (in Julia or Python) to compute $e^x$ directly from the Taylor series definition:
$$
e^x = 1 + x + \frac{x^2}{2} + \cdots + \frac{x^n}{n!} + \cdots \, .
$$
(in the default `Float64`/`float` precision … no fair using arbitrary-precision arithmetic).

1. Explain how you can compute each term in the series from the preceding term.  (Not only is this more efficient, but it also helps avoid overflow compared to the naive approach where you compute $x^n$ and $n!$ *separately* and then divide them.)

2. Explain how you decided how many terms to sum, to make reasonably sure that the omitted terms have a negligible contribution.  (Your method should depend on $x$.  Does not need a rigorous argument, just a reasonable explanation.)

3. Check that `myexp(100.0)` gives a small *relative* error ($< 10^{-14}$) compared to `exp(100.0)` in Julia or `math.exp(100.0)` in Python), even though your `myexp(100.0) - exp(100.0)` is probably huge.

4. Explain why `myexp(-100.0)` gives a completely wrong result, no matter how many terms you include in the sum!

## Problem 3 (5+5 points)

Write a function $L4(x,y)$ in Julia or Python that computes the "$L_4$ norm" $L4(x,y) = (x^4 + y^4)^{1/4}$ of two real (floating-point) scalars $x$ and $y$.

1. If you implement this in the most straightforward way, directly from the formula above, does your code give an accurate answer for `L4(1e-100, 0.0)`?  What about for `L4(1e100, 0.0)`?  Why or why not?
2. *Fix* your code so that it gives an accurate answer (a *small relative error* close to machine precision)) for all floating-point inputs $x$ and $y$ (including the case in the previous part). (No fair resorting to higher-precision arithmetic!)